In [1]:
import pandas as pd
import numpy as np
import gc

import os
import sys
sys.path.append("../")

from scipy.sparse import csc_matrix
from scipy.sparse import coo_matrix


from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import MinMaxScaler


import Tool.utils as utils
import Tool.config as config
from Feature import one_hot_feature

from scipy.sparse import hstack
from scipy.sparse import vstack

In [2]:
data = pd.read_pickle(config.data_prefix_path + 'data.pkl')

In [3]:
object_features = ["predict_category_1","predict_category_2","predict_category_0",
                    "predict_property_0","predict_property_1","predict_property_2",
                    "property_1","property_0","property_2",
                    "category_1","category_0","category_2",
                    'category_cross_0', 'category_cross_1', 'category_cross_2',
                    'hour_and_category_1',
                    'user_gender_id','user_occupation_id',]

In [4]:
features = [c for c in data.columns if c not in ['is_trade', 'instance_id','index',
                                            'context_id', 'time', 'day','context_timestamp',
                                            'property_list','category_list','property_predict_list','category_predict_list',
                                            'item_category_list', 'item_property_list', 'predict_category_property',
                                            'user_id','item_id','item_brand_id','item_city_id','shop_id',
                                            ]
            and c not in object_features]
target = ['is_trade']

In [5]:
train = data[data.is_trade.notnull()]
test = data[data.is_trade.isnull()]
del data
gc.collect()

0

## LR线下验证

尝试过在one_hot特征上再加入原始特征，讲道理效果会更好   
但是模型跑不出来(非稀疏矩阵LR速度太慢,也有可能是我忘了scaler....)
***

In [6]:
X_train = train[train.day < 24]
X_val = train[train.day == 24]

del train,test
gc.collect()

0

In [7]:
train_Y = X_train['is_trade'].values.ravel()
val_Y = X_val['is_trade'].values.ravel()

X_train = np.isnan(X_train[features].values)
X_val = np.isnan(X_val[features].values)

In [8]:
# Scaler
scaler  = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype bool was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:
# 读取one_hot特征
train_X,val_X = one_hot_feature.get_one_hot_data('val')

# 将one_hot特征和原始特征拼接
print("")
print("数据拼接")
train_X = hstack([train_X,X_train])
val_X = hstack([val_X,X_val])

del X_train,X_val
gc.collect()

获取类别特征one_hot线下验证数据
训练集长度: 420627
验证集长度：57405
获取xgboost特征one_hot线下验证数据
训练集长度: 420627
测试集长度：57405

数据拼接


73

In [16]:
lr = LogisticRegression(n_jobs=6,C=0.00045)
lr.fit(train_X, train_Y)
    
y_train_pred = lr.predict_proba(train_X)[:,1]
train_log = log_loss(train_Y,y_train_pred)
    
y_val_pred = lr.predict_proba(val_X)[:,1]
val_log = log_loss(val_Y,y_val_pred)
    
print(train_log)
print(val_log)
print(roc_auc_score(val_Y,y_val_pred))

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(self.n_jobs))


0.080240328679
0.0799438203862
0.717387927204


## LR线上提交

In [6]:
X_train = np.isnan(train[features].values)
X_test = np.isnan(test[features].values)

In [7]:
# Scaler
scaler  = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype bool was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
train_X,test_X = one_hot_feature.get_one_hot_data('train')
train_Y = train['is_trade'].values.ravel()

print("")
print("数据拼接")
train_X = hstack([train_X,X_train])
test_X = hstack([test_X,X_test])

del train,X_train,X_test
gc.collect()

获取类别特征one_hot线上提交数据
训练集长度: 478032
测试集长度：42888
获取xgboost特征one_hot线上提交数据
训练集长度: 478032
测试集长度：42888

数据拼接


48

In [10]:
lr = LogisticRegression(n_jobs=6,C=0.00045)
lr.fit(train_X, train_Y)

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=0.00045, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=6, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [11]:
test['predicted_score'] = lr.predict_proba(test_X)[:,1]

In [12]:
test[['instance_id', 'predicted_score']].to_csv(config.data_prefix_path + 'sub0421_xgboost_LR.txt',sep=" ",index=False)

## OOF

In [6]:
X_train = np.isnan(train[features].values)
scaler  = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

train_Y = train['is_trade'].values.ravel()

del test
gc.collect()

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype bool was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


0

In [7]:
# 读取one_hot特征
train_X,val_X = one_hot_feature.get_one_hot_data('val')
train_X = vstack([train_X,val_X])

# 将one_hot特征和原始特征拼接
print("")
print("数据拼接")
train_X = coo_matrix.tocsr(hstack([train_X,X_train]))

del X_train,val_X
gc.collect()

获取类别特征one_hot线下验证数据
训练集长度: 420627
验证集长度：57405
获取xgboost特征one_hot线下验证数据
训练集长度: 420627
测试集长度：57405

数据拼接


24

In [8]:
from sklearn.model_selection import KFold
folds=KFold(n_splits=4,random_state=2018,shuffle=True)

pred = np.zeros(train_X.shape[0])

In [10]:
for n_fold,(trn_idx,val_idx) in enumerate(folds.split(train_X,train_Y)):
    
    lr = LogisticRegression(n_jobs=6,C=0.00045)
    lr.fit(train_X[trn_idx], train_Y[trn_idx])

    pred[val_idx] = lr.predict_proba(train_X[val_idx])[:,1]
    loss = log_loss(train_Y[val_idx],pred[val_idx])
    
    print("\t Fold %d : %.6f" % (n_fold+1, loss))
    
train['is_trade'+'_oof'] = pred

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(self.n_jobs))


	 Fold 1 : 0.081061
	 Fold 2 : 0.082533
	 Fold 3 : 0.081702
	 Fold 4 : 0.081844


In [11]:
# 保存结果,用作stacking
train[['instance_id', 'is_trade', 'is_trade'+'_oof']].to_csv(
    config.data_prefix_path + 'xgboost_LR_oof.txt',sep=" ",index=False
    )